Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [2]:
import pandas as pd

## Import .csv from API calls
STATIONS= pd.read_csv('../data/city_bikes.csv')
YELP = pd.read_csv('../data/Yelp_location_results.csv')
FS = pd.read_csv('../data/FS_location_results.csv')

In [3]:
### EXPLORATION AND TESTING OF PRIMARY KEY BETWEEN TABLES. 

#Architecture 
# - STATIONS will be the LEFT TABLE as it is the 'master' list of station id and the lookup key.
# - LAT/LONG for the STATION will be the primary key
# - JOIN dataframes on LAT/LONG. Anticipate there will be multiple results per station id (1-many)


# STATIONS.drop_duplicates(subset='lat_long',keep="first").count()

## Add primary key to Yelp based on the Lat/Long of the associated station coords
## Convert it to string or it will just ADD them! doh

YELP['lat_long']=YELP['latitude'].astype(str) +","+ YELP['longitude'].astype(str)
YELP['source']="yelp"
YELP_df = YELP[['name','station_id','rating','price','lat_long','distance','source']]
YELP_df
## Confirm that generated pkey ('lat_long' can be used to join tables. 
## Postive outcome: use outpt of concatenated field as input to filter the Station list. 
## Result : success. 
## Conclusion: Use the generated 'lat_long' as a primary key to join tables. 

# Sample Lat_long from Yelp='49.262487,-123.114397'
# 
# STATIONS_filter = STATIONS[STATIONS['lat_long']=='49.262487,-123.114397']
# STATIONS_filter




,name,station_id,rating,price,lat_long,distance,source
0,La Taqueria Pinche Taco Shop,7a19c49f486d7c0c02b3685d7b240448,4.0,$$,"49.262487,-123.114397",169.517456,yelp
1,Saku,7a19c49f486d7c0c02b3685d7b240448,4.5,$$,"49.262487,-123.114397",178.845344,yelp
2,Uma Sushi,7a19c49f486d7c0c02b3685d7b240448,4.5,$$$,"49.262487,-123.114397",152.157897,yelp
3,Hokkaido Ramen Santouka,7a19c49f486d7c0c02b3685d7b240448,4.0,$$,"49.262487,-123.114397",191.044234,yelp
4,iDen & Quan Ju De Beijing Duck House,7a19c49f486d7c0c02b3685d7b240448,3.5,NaN,"49.262487,-123.114397",263.641336,yelp
...,...,...,...,...,...,...,...
12115,A&W,cc25ae4f093b33ba0afd1dbc0dd20324,3.5,$,"49.265442,-123.187738",321.985495,yelp
12116,Dairy Queen / Orange Julius,cc25ae4f093b33ba0afd1dbc0dd20324,4.5,NaN,"49.265442,-123.187738",987.901958,yelp
12117,Freshslice Pizza,cc25ae4f093b33ba0afd1dbc0dd20324,3.5,$,"49.265442,-123.187738",333.363870,yelp
12118,720 Sweets,cc25ae4f093b33ba0afd1dbc0dd20324,3.5,$,"49.265442,-123.187738",782.192237,yelp


In [4]:
### CREATE FOREIGN KEYS IN YELP AND FS DATAFRAMES

## Eliminate extra fields in STATIONS
STATIONS_abr = STATIONS[['lat_long','name','free_bikes']]
STATIONS_abr


YELP['lat_long']=YELP['latitude'].astype(str) +","+ YELP['longitude'].astype(str)
YELP['source']="yelp"
YELP_df = YELP[['name','station_id','rating','price','lat_long','distance','source','id']]
YELP_df
## QA - CHECK - 12,120 records. Lat_long Generated. 

FS['lat_long']=FS['latitude'].astype(str) +","+ FS['longitude'].astype(str)
FS['source']="FS"
FS_df = FS[['name','station_id','rating','price','lat_long','distance','source','fsq_id']]
FS_df

## QA - Observation - FS may have limited the results as there is an curious consistency about the max number of results per STATION
## todo: investigate why there might be a max of then bars per stationid in the FS results 
##FS.groupby('station_id')['name'].count().sort_values(ascending=False)


,name,station_id,rating,price,lat_long,distance,source,fsq_id
0,Rogue Kitchen & Wetbar,7a19c49f486d7c0c02b3685d7b240448,8.2,3.0,"49.262487,-123.114397",250,FS,4f60fc8be4b0a9d090a1f850
1,33 Acres Brewing Co,7a19c49f486d7c0c02b3685d7b240448,9.4,2.0,"49.262487,-123.114397",674,FS,51b201d27dd249ae714ba728
2,The Juice Truck,7a19c49f486d7c0c02b3685d7b240448,8.8,1.0,"49.262487,-123.114397",779,FS,537e3ec8498ec730b9966b97
3,Biercraft Bistro,7a19c49f486d7c0c02b3685d7b240448,8.1,3.0,"49.262487,-123.114397",744,FS,4cbf89f200d8370403c8445c
4,Faculty Brewing Co,7a19c49f486d7c0c02b3685d7b240448,8.3,2.0,"49.262487,-123.114397",994,FS,55d97956498e2c299b5830c0
...,...,...,...,...,...,...,...,...
2259,The Fringe Cafe,cc25ae4f093b33ba0afd1dbc0dd20324,5.8,2.0,"49.265442,-123.187738",955,FS,4aa7f754f964a520484e20e3
2260,Coppertank Grill,cc25ae4f093b33ba0afd1dbc0dd20324,5.7,1.0,"49.265442,-123.187738",941,FS,4aa6a91df964a520864a20e3
2261,Good Co. Kits,cc25ae4f093b33ba0afd1dbc0dd20324,NaN,1.0,"49.265442,-123.187738",791,FS,6457401321f70230178ea1ea
2262,Castaway Bar and Kitchen,cc25ae4f093b33ba0afd1dbc0dd20324,NaN,2.0,"49.265442,-123.187738",809,FS,5d29827cf5c8330023780aff


# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [34]:
import sqlite3
from sqlite3 import Error


def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("connection to sq3 db successful")
    except Error as e:
        print(f"The error '{e}' happened")

    return connection

## Create a CONNECTION between python and the sqlite file 'sm_app'
connection = create_connection('../data/vancouver_bikes')

connection to sq3 db successful


In [19]:

## Create TABLES

## Tables: 
    ## - vancouver_stations
    ## - yelp_data

## 1. Confirm list of Fields for tables to create fields in DB accordingly. 
# print(STATIONS.info())
print(YELP_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12120 entries, 0 to 12119
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        12120 non-null  object 
 1   station_id  12120 non-null  object 
 2   rating      12120 non-null  float64
 3   price       10169 non-null  object 
 4   lat_long    12120 non-null  object 
 5   distance    12120 non-null  float64
 6   source      12120 non-null  object 
 7   id          12120 non-null  object 
dtypes: float64(2), object(6)
memory usage: 757.6+ KB
None


In [32]:
### DEFINE FUNCTION FOR CREATE/DELETE IN DB 

def execute_CD_qry(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"Computer reported '{e}' error")
        
        
def execute_station_update_qry(connection, query):
    cursor = connection.cursor()
    try:
        for index, row in STATIONS.iterrows():
            try:
                # Define your INSERT query
                query
                
                # Get the values from the DataFrame row
                values = (
                    row['empty_slots'],
                    row['free_bikes'],
                    row['id'],
                    row['latitude'],
                    row['longitude'],
                    row['name'],
                    row['timestamp'],
                    row['extra_ebikes'],
                    row['extra_has_ebikes'],
                    row['lat_long'],
                    row['extra_last_updated']
                )
    
                # Execute the INSERT query with the row values
                cursor.execute(query, values)
            except Error as e:
                print(f"Error inserting row: {e}")
                connection.rollback()  # Rollback the transaction in case of an error
        connection.commit()
    except Error as e:
        print(f"Error connecting to the database: {e}")
    finally:
        if connection:
            cursor.close()
            connection.close()

In [29]:
### Create TABLE: vancouver_stations

''' PANDA DATA TYPE - STATIONS
 0   empty_slots         245 non-null    int64  
 1   free_bikes          245 non-null    int64  
 2   id                  245 non-null    object 
 3   latitude            245 non-null    float64
 4   longitude           245 non-null    float64
 5   name                245 non-null    object 
 6   timestamp           245 non-null    object 
 7   extra_ebikes        245 non-null    int64  
 8   extra_has_ebikes    245 non-null    bool   
 9   extra_last_updated  245 non-null    int64  
 10  lat_long            245 non-null    object '''

'''MAP TO SQLITE TYPE
empty_slots	        INTEGER
free_bikes	        INTEGER
id	                TEXT
latitude	        REAL
longitude	        REAL
name	            TEXT
timestamp	        TEXT (assuming it's a string representation of date-time)
extra_ebikes	    INTEGER
extra_has_ebikes	INTEGER (0 for False, 1 for True)
extra_last_updated	INTEGER
lat_long	        TEXT
'''
###########################################
'''PANDA DATA TYPE - YELP
    name        12120 non-null  object 
 1   station_id  12120 non-null  object 
 2   rating      12120 non-null  float64
 3   price       10169 non-null  object 
 4   lat_long    12120 non-null  object 
 5   distance    12120 non-null  float64
 6   source      12120 non-null  object 
 7   id          12120 non-null  object
 '''
'''MAP TO SQLITE TYPE
name  TEXT 
station_id  TEXT 
rating  float8
price  float8 
lat_long  TEXT 
distance  float8
source TEXT 
id TEXT'''

#### QUERIES: CREATE (DELETE) DATA ELEMENTS ######

create_vancouver_stations_tble = '''
    CREATE TABLE IF NOT EXISTS vancouver_stations (
            empty_slots INTEGER,
            free_bikes INTEGER,
            id TEXT,
            latitude REAL,
            longitude REAL,
            name TEXT,
            timestamp TEXT ,
            extra_ebikes INTEGER,
            extra_has_ebikes INTEGER,
            extra_last_updated INTEGER,
            lat_long TEXT PRIMARY KEY
             );'''

create_yelp_tble = '''
    CREATE TABLE IF NOT EXISTS yelp_data (
        name TEXT, 
        station_id TEXT, 
        rating float8,
        price float8,
        lat_long TEXT, 
        distance float8,
        source TEXT, 
        id TEXT);'''


## QUERY - INSERT RECORDS USING DATAFRAME

insert_station_records = """
        INSERT INTO vancouver_stations (
            empty_slots,
            free_bikes,
            id,
            latitude,
            longitude,
            name,
            timestamp,
            extra_ebikes,
            extra_has_ebikes,
            extra_last_updated,
            lat_long) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

insert_yelp_records = """
        INSERT INTO yelp_data (
            name, 
            station_id, 
            rating,
            price,
            lat_long, 
            distance,
            source, 
            id) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """

In [21]:
## CALL DATABASE FUNCTIONS: 

## CREATE TABLES:
execute_CD_qry(connection, create_vancouver_stations_tble)
execute_CD_qry(connection,create_yelp_tble)

Query executed successfully
Query executed successfully


Look at the data before and after the join to validate your data.

In [36]:
execute_station_update_qry(connection,insert_station_records)

ProgrammingError: Cannot operate on a closed database.

In [31]:
STATIONS

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra_ebikes,extra_has_ebikes,extra_last_updated,lat_long
0,5,29,7a19c49f486d7c0c02b3685d7b240448,49.262487,-123.114397,10th & Cambie,2023-09-21 19:03:51.469000,6,True,1695322925,"49.262487,-123.114397"
1,12,4,32603a87cfca71d0f7dfa3513bad69d5,49.274566,-123.121817,Yaletown-Roundhouse Station,2023-09-21 19:03:51.456000,0,True,1695322945,"49.274566,-123.121817"
2,13,13,6d42fa40360f9a6b2bf641c7b8bb2862,49.279764,-123.110154,Dunsmuir & Beatty,2023-09-21 19:03:51.457000,1,True,1695322662,"49.279764,-123.110154"
3,11,5,66f873d641d448bd1572ab086665a458,49.260599,-123.113504,12th & Yukon (City Hall),2023-09-21 19:03:51.458000,4,True,1695322780,"49.260599,-123.113504"
4,9,7,485d4d24c803cfde829ab89699fed833,49.264215,-123.117772,8th & Ash,2023-09-21 19:03:51.477000,4,True,1695322949,"49.264215,-123.117772"
...,...,...,...,...,...,...,...,...,...,...,...
240,34,6,a74744ce4bb7ea2aa9f406ac8bff95d8,49.280977,-123.035969,PNE - Hastings & Windermere,2023-09-21 19:03:51.275000,2,True,1695322761,"49.280977,-123.035969"
241,8,11,5699b40126e10c2f68eefc3eb18ff3a1,49.270783,-123.141564,1st & Fir,2023-09-21 19:03:51.247000,1,True,1695322960,"49.270783,-123.141564"
242,15,7,34fd37d12eb989b49518ef53941ee3ff,49.264019,-123.209176,Sasamat & 10th,2023-09-21 19:03:51.199000,3,True,1695322610,"49.264019,-123.209176"
243,18,2,b7dd37a7dd668d6d10024b7f18acc438,49.265800,-123.205960,Trimble & 8th,2023-09-21 19:03:51.246000,1,True,1695322940,"49.2658,-123.20596"
